In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
from gnomad.utils.plotting import *
from typing import Set, Tuple

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201119-1711-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [2]:
synonymous="s3a://DDD-ELGH-UKBB-exomes/vep/vep_consequences_worst_synonymous.tsv"

In [20]:
ht_synonymous=hl.import_table(synonymous,impute=True, no_header=True)
#ht = ht.key_by(
#    locus=hl.locus(ht.chromosome, ht.position, reference_genome='GRCh37'), 
#    alleles=[ht.reference, ht.mutation])

2020-11-19 17:28:55 Hail: INFO: Reading table to impute column types
2020-11-19 17:28:56 Hail: INFO: Finished type imputation
  Loading column 'f0' as type 'str' (imputed)
  Loading column 'f1' as type 'int32' (imputed)
  Loading column 'f2' as type 'str' (imputed)
  Loading column 'f3' as type 'str' (imputed)
  Loading column 'f4' as type 'str' (imputed)


In [23]:
ht_synonymous=ht_synonymous.annotate(chrom=ht_synonymous.f0)
ht_synonymous=ht_synonymous.annotate(position=ht_synonymous.f1)
ht_synonymous=ht_synonymous.annotate(ref=ht_synonymous.f2)
ht_synonymous=ht_synonymous.annotate(alt=ht_synonymous.f3)
ht_synonymous=ht_synonymous.annotate(consequence=ht_synonymous.f4)
ht1=ht_synonymous.select(ht_synonymous.chrom,ht_synonymous.position,ht_synonymous.ref,ht_synonymous.alt,ht_synonymous.consequence)
#t = t.annotate(sample = t.f0.split("_")[1]).key_by('sample')t = t.annotate(sample = t.f0.split("_")[1]).key_by('sample')

In [24]:
ht1.show()

chrom,position,ref,alt,consequence
str,int32,str,str,str
"""chr1""",69081,"""G""","""C""","""synonymous_variant"""
"""chr1""",69270,"""A""","""G""","""synonymous_variant"""
"""chr1""",69360,"""C""","""T""","""synonymous_variant"""
"""chr1""",69366,"""T""","""G""","""synonymous_variant"""
"""chr1""",69369,"""G""","""A""","""synonymous_variant"""
"""chr1""",69393,"""T""","""C""","""synonymous_variant"""
"""chr1""",69438,"""T""","""C""","""synonymous_variant"""
"""chr1""",69453,"""G""","""A""","""synonymous_variant"""
"""chr1""",69462,"""C""","""T""","""synonymous_variant"""


In [25]:
ht1 = ht1.key_by(
    locus=hl.locus(ht1.chrom, ht1.position, reference_genome='GRCh38'), 
    alleles=[ht1.ref, ht1.alt])

In [26]:
ht1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'chrom': str 
    'position': int32 
    'ref': str 
    'alt': str 
    'consequence': str 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [28]:
ht1.write(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_synonymous_variants.ht')

2020-11-19 17:31:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-11-19 17:31:48 Hail: INFO: wrote table with 1399620 rows in 100 partitions to file:///home/ubuntu/data/tmp/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_synonymous_variants.ht


In [29]:
ht1=hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_synonymous_variants.ht')

In [30]:
run_hash="91b132aa"

ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_sanger_cohorts_DENOVO_family_stats.ht')

In [31]:
ht=ht.annotate(consequence=ht1[ht.key].consequence)

In [32]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32, 
        QD: float64, 
        MQRankSum: float64, 
        SOR: float64, 
        ReadPosRankSum: float64, 
        FS: float64, 
        DP: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': 

In [33]:
 ht.write(f'{tmp_dir}/{run_hash}_rf_result_sanger_cohorts_DENOVO_family_stats_SYNONYMOUS_annotation.ht', overwrite=True)

2020-11-19 17:34:56 Hail: INFO: wrote table with 13669739 rows in 500 partitions to hdfs://spark-master:9820//91b132aa_rf_result_sanger_cohorts_DENOVO_family_stats_SYNONYMOUS_annotation.ht


In [34]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_sanger_cohorts_DENOVO_family_stats_SYNONYMOUS_annotation.ht')

In [35]:
ht.show()

locus,alleles,a_index,was_split,InbreedingCoeff,variant_type,allele_type,n_alt_alleles,was_mixed,has_star,QD,MQRankSum,SOR,ReadPosRankSum,FS,DP,hapmap,omni,mills,kgp_phase1_hc,transmitted_singleton,fail_hard_filters,ac_raw,feature_imputed.a_index,feature_imputed.n_alt_alleles,feature_imputed.QD,feature_imputed.MQRankSum,feature_imputed.SOR,feature_imputed.ReadPosRankSum,feature_imputed.FS,feature_imputed.DP,tp,fp,rf_train,rf_label,rf_test,rf_probability,rf_prediction,de_novo_data,family_stats,consequence
locus<GRCh38>,array<str>,int32,bool,float32,str,str,int32,bool,bool,float64,float64,float64,float64,float64,int32,bool,bool,bool,bool,bool,bool,int64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,bool,"dict<str, float64>",str,"array<struct{id: str, prior: float64, proband: struct{s: str}, father: struct{s: str}, mother: struct{s: str}, proband_entry: struct{AD: array<int32>, DP: int32, GQ: int32, GT: call, MIN_DP: int32, PGT: call, PID: str, PL: array<int32>, PS: int32, RGQ: int32, SB: array<int32>}, father_entry: struct{AD: array<int32>, DP: int32, GQ: int32, GT: call, MIN_DP: int32, PGT: call, PID: str, PL: array<int32>, PS: int32, RGQ: int32, SB: array<int32>}, mother_entry: struct{AD: array<int32>, DP: int32, GQ: int32, GT: call, MIN_DP: int32, PGT: call, PID: str, PL: array<int32>, PS: int32, RGQ: int32, SB: array<int32>}, is_female: bool, p_de_novo: float64, confidence: str}>","array<struct{mendel: struct{errors: int64}, tdt: struct{t: int64, u: int64, chi_sq: float64, p_value: float64}, unrelated_qc_callstats: struct{AC: array<int32>, AF: array<float64>, AN: int32, homozygote_count: array<int32>}, meta: dict<str, str>}>",str
chr1:12938,"[""GCAAA"",""G""]",1,false,1.00e+00,"""indel""","""del""",1,false,false,2.87e+01,0.00e+00,2.30e+00,0.00e+00,0.00e+00,3591,NA,NA,NA,NA,false,true,2,false,false,false,true,false,true,false,false,NA,true,true,"""FP""",false,"{""FP"":2.41e-01},""TP"":7.59e-01}}","""TP""",NA,"[((0),(NA,NA,NA,NA),([4198,2],[1.00e+...",NA
chr1:13024,"[""G"",""A""]",1,false,1.00e+00,"""snv""","""snv""",1,false,false,2.72e+01,0.00e+00,3.26e+00,0.00e+00,0.00e+00,13955,NA,NA,NA,NA,false,true,2,false,false,false,true,false,true,false,false,NA,true,true,"""FP""",false,"{""FP"":2.84e-01},""TP"":7.16e-01}}","""TP""",NA,"[((1),(NA,NA,NA,NA),([12992,0],[1.00e...",NA
chr1:13087,"[""A"",""G""]",1,false,8.00e-01,"""snv""","""snv""",1,false,false,7.23e+00,-1.36e-01,5.95e-01,1.69e+00,0.00e+00,39714,NA,NA,NA,NA,false,false,5,false,false,false,false,false,false,false,false,NA,false,NA,NA,false,"{""FP"":4.79e-01},""TP"":5.21e-01}}","""TP""",NA,"[((1),(NA,NA,NA,NA),([30709,3],[1.00e...",NA
chr1:13116,"[""T"",""C""]",1,false,1.00e+00,"""snv""","""snv""",1,false,false,2.46e+01,0.00e+00,1.02e+00,0.00e+00,0.00e+00,41952,NA,NA,NA,NA,false,true,2,false,false,false,true,false,true,false,false,NA,true,true,"""FP""",false,"{""FP"":2.05e-01},""TP"":7.95e-01}}","""TP""",NA,"[((0),(NA,NA,NA,NA),([31130,2],[1.00e...",NA
chr1:13130,"[""C"",""T""]",1,false,1.00e+00,"""snv""","""snv""",1,false,false,2.74e+01,0.00e+00,1.18e+00,0.00e+00,0.00e+00,31819,NA,NA,NA,NA,false,false,2,false,false,false,true,false,true,false,false,NA,false,NA,NA,false,"{""FP"":2.03e-01},""TP"":7.97e-01}}","""TP""",NA,"[((0),(NA,NA,NA,NA),([27456,2],[1.00e...",NA
chr1:13151,"[""G"",""C""]",1,false,1.00e+00,"""snv""","""snv""",1,false,false,1.65e+01,0.00e+00,2.30e+00,0.00e+00,0.00e+00,36547,NA,NA,NA,NA,false,true,2,false,false,false,true,false,true,false,false,NA,true,true,"""FP""",false,"{""FP"":2.87e-01},""TP"":7.13e-01}}","""TP""",NA,"[((0),(NA,NA,NA,NA),([31880,2],[1.00e...",NA
chr1:13164,"[""G"",""A""]",1,false,1.00e+00,"""snv""","""snv""",1,false,false,1.95e+01,0.00e+00,1.18e+00,0.00e+00,0.00e+00,44960,NA,NA,NA,NA,false,true,2,false,false,false,true,false,true,false,false,NA,true,true,"""FP""",false,"{""FP"":3.06e-01},""TP"":6.94e-01}}","""TP""",NA,"[((0),(NA,NA,NA,NA),([35810,2],[1.00e...",NA
chr1:13176,"[""G"",""T""]",1,false,6.67e-01,"""snv""","""snv"""

In [36]:
ht.consequence.summarize()

Non-missing,552327 (4.04%)
Missing,13117412 (95.96%)
Min Size,18
Max Size,86
Mean Size,21.86
Sample Values,"['synonymous_variant', 'synonymous_variant', 'synonymous_variant', 'synonymous_variant', 'synonymous_variant']"
